In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import joblib
# Load the dataset
file_path = "Wellbeing_and_lifestyle_data_Kaggle.csv"  # Adjust the path if needed
df = pd.read_csv(file_path)

# Drop unnecessary columns
df.drop(columns=['Timestamp'], inplace=True)

# Handle missing values
for col in df.columns:
    if df[col].dtype == 'object':
        df[col].fillna(df[col].mode()[0], inplace=True)  # Fill categorical with mode
    else:
        df[col].fillna(df[col].median(), inplace=True)  # Fill numerical with median

# Convert categorical features to numerical using Label Encoding
label_encoders = {}
categorical_features = ['DAILY_STRESS', 'AGE', 'GENDER']

for col in categorical_features:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# Identify and remove outliers using the IQR method
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1
df = df[~((df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR))).any(axis=1)]
# Identify numerical features
numerical_features = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
# Apply StandardScaler
scaler = StandardScaler()
df[numerical_features] = scaler.fit_transform(df[numerical_features])
# Save the scaler for later use
joblib.dump(scaler, "scaler.pkl")

print("Preprocessing complete. Scaler saved as 'scaler.pkl'.")
# Save preprocessed dataset
df.to_csv("processed_wellbeing_data.csv", index=False)

# Display completion message
print("Preprocessing complete. Processed dataset saved as 'processed_wellbeing_data.csv'.")


<ipython-input-1-4025a155d312>:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)  # Fill numerical with median
<ipython-input-1-4025a155d312>:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(val

Preprocessing complete. Scaler saved as 'scaler.pkl'.
Preprocessing complete. Processed dataset saved as 'processed_wellbeing_data.csv'.


✅ composite scores <br>
SOCIAL_ACTIVITY_SCORE (based on social interactions).<br>
WORK_PRODUCTIVITY_SCORE (task completion & achievements).<br>
SELF_CARE_SCORE (sleep, meditation, personal time).<br>
STRESS_IMPACT (daily stress impact).<br>
✅binary target variables:<br>

COGNITIVE_OVERLOAD: 1 if stress levels are high.<br>
SOCIAL_ENGAGEMENT_NEEDS: 1 if social interaction is low.<br>
WORK_LIFE_BALANCE_ADJUST: 1 if work-life balance is poor.

In [ ]:
# Feature Engineering: Creating meaningful features
df['SOCIAL_ACTIVITY_SCORE'] = df['PLACES_VISITED'] + df['SOCIAL_NETWORK'] + df['SUPPORTING_OTHERS']
df['WORK_PRODUCTIVITY_SCORE'] = df['ACHIEVEMENT'] + df['TODO_COMPLETED'] + df['PERSONAL_AWARDS']
df['SELF_CARE_SCORE'] = df['SLEEP_HOURS'] + df['TIME_FOR_PASSION'] + df['WEEKLY_MEDITATION']
df['STRESS_IMPACT'] = df['DAILY_STRESS'] + df['DAILY_SHOUTING'] - df['LIVE_VISION']

# Binary Labels for Cognitive Overload (1: Overloaded, 0: Not Overloaded)
df['COGNITIVE_OVERLOAD'] = np.where(df['DAILY_STRESS'] > df['DAILY_STRESS'].median(), 1, 0)

# Social Engagement Needs (1: Needs more engagement, 0: Well engaged)
df['SOCIAL_ENGAGEMENT_NEEDS'] = np.where(df['SOCIAL_NETWORK'] < df['SOCIAL_NETWORK'].median(), 1, 0)

# Work-Life Balance Classification (1: Needs Adjustment, 0: Balanced)
df['WORK_LIFE_BALANCE_ADJUST'] = np.where(df['WORK_LIFE_BALANCE_SCORE'] < df['WORK_LIFE_BALANCE_SCORE'].median(), 1, 0)

# Save the new dataset with engineered features
df.to_csv("engineered_wellbeing_data.csv", index=False)

print("Feature engineering complete. Dataset saved as 'engineered_wellbeing_data.csv'.")


Feature engineering complete. Dataset saved as 'engineered_wellbeing_data.csv'.


Machine Learning Model for Cognitive Overload Prediction<bR>
We'll use:<br>
 ✅ Logistic Regression as a baseline model.<br>
✅ Random Forest Classifier for improved accuracy.<br>
✅ XGBoost Classifier for optimized performance.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Load the dataset with engineered features
df = pd.read_csv("engineered_wellbeing_data.csv")

# Select features and target variable
features = ['SOCIAL_ACTIVITY_SCORE', 'WORK_PRODUCTIVITY_SCORE', 'SELF_CARE_SCORE', 'STRESS_IMPACT']
X = df[features]
y = df['COGNITIVE_OVERLOAD']  # Target variable (1: Overloaded, 0: Not Overloaded)

# Split the data into training and testing sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Logistic Regression Model (Baseline)
log_model = LogisticRegression()
log_model.fit(X_train, y_train)
log_preds = log_model.predict(X_test)
log_acc = accuracy_score(y_test, log_preds)
print("Logistic Regression Accuracy:", log_acc)

# Train Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_preds = rf_model.predict(X_test)
rf_acc = accuracy_score(y_test, rf_preds)
print("Random Forest Accuracy:", rf_acc)

# Train XGBoost Classifier
xgb_model = XGBClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
xgb_model.fit(X_train, y_train)
xgb_preds = xgb_model.predict(X_test)
xgb_acc = accuracy_score(y_test, xgb_preds)
print("XGBoost Accuracy:", xgb_acc)

# Print Model Performance
print("\nClassification Report (Random Forest):\n", classification_report(y_test, rf_preds))
print("\nClassification Report (XGBoost):\n", classification_report(y_test, xgb_preds))

# Save the trained models
import joblib
joblib.dump(rf_model, "random_forest_cognitive_overload.pkl")
joblib.dump(xgb_model, "xgboost_cognitive_overload.pkl")

print("Model training complete. Models saved successfully.")


Logistic Regression Accuracy: 0.7843791722296395
Random Forest Accuracy: 0.920894526034713
XGBoost Accuracy: 0.9479305740987984

Classification Report (Random Forest):
               precision    recall  f1-score   support

           0       0.94      0.95      0.94      2125
           1       0.88      0.84      0.86       871

    accuracy                           0.92      2996
   macro avg       0.91      0.90      0.90      2996
weighted avg       0.92      0.92      0.92      2996


Classification Report (XGBoost):
               precision    recall  f1-score   support

           0       0.96      0.97      0.96      2125
           1       0.92      0.90      0.91       871

    accuracy                           0.95      2996
   macro avg       0.94      0.93      0.94      2996
weighted avg       0.95      0.95      0.95      2996

Model training complete. Models saved successfully.


Machine Learning Model for Social Engagement Prediction<br>
We'll use: <br>✅ Logistic Regression as a baseline model.<br>
✅ Random Forest Classifier for better accuracy.<br>
✅ XGBoost Classifier for optimized performance.<br>

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Load the dataset with engineered features
df = pd.read_csv("engineered_wellbeing_data.csv")

# Select features and target variable
features = ['SOCIAL_ACTIVITY_SCORE', 'WORK_PRODUCTIVITY_SCORE', 'SELF_CARE_SCORE', 'STRESS_IMPACT']
X = df[features]
y = df['SOCIAL_ENGAGEMENT_NEEDS']  # Target variable (1: Needs Engagement, 0: Well Engaged)

# Split the data into training and testing sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Logistic Regression Model (Baseline)
log_model = LogisticRegression()
log_model.fit(X_train, y_train)
log_preds = log_model.predict(X_test)
log_acc = accuracy_score(y_test, log_preds)
print("Logistic Regression Accuracy:", log_acc)

# Train Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_preds = rf_model.predict(X_test)
rf_acc = accuracy_score(y_test, rf_preds)
print("Random Forest Accuracy:", rf_acc)

# Train XGBoost Classifier
xgb_model = XGBClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
xgb_model.fit(X_train, y_train)
xgb_preds = xgb_model.predict(X_test)
xgb_acc = accuracy_score(y_test, xgb_preds)
print("XGBoost Accuracy:", xgb_acc)

# Print Model Performance
print("\nClassification Report (Random Forest):\n", classification_report(y_test, rf_preds))
print("\nClassification Report (XGBoost):\n", classification_report(y_test, xgb_preds))

# Save the trained models
import joblib
joblib.dump(rf_model, "random_forest_social_engagement.pkl")
joblib.dump(xgb_model, "xgboost_social_engagement.pkl")

print("Model training complete. Models saved successfully.")


Logistic Regression Accuracy: 0.784712950600801
Random Forest Accuracy: 0.9472630173564753
XGBoost Accuracy: 0.9355807743658211

Classification Report (Random Forest):
               precision    recall  f1-score   support

           0       0.95      0.95      0.95      1692
           1       0.94      0.94      0.94      1304

    accuracy                           0.95      2996
   macro avg       0.95      0.95      0.95      2996
weighted avg       0.95      0.95      0.95      2996


Classification Report (XGBoost):
               precision    recall  f1-score   support

           0       0.94      0.94      0.94      1692
           1       0.93      0.93      0.93      1304

    accuracy                           0.94      2996
   macro avg       0.93      0.93      0.93      2996
weighted avg       0.94      0.94      0.94      2996

Model training complete. Models saved successfully.


Machine Learning Model for Work-Life Balance Adjustment<br>
We'll use: <br>✅ Logistic Regression as a baseline model.<br>
✅ Random Forest Classifier for better accuracy.<br>
✅ XGBoost Classifier for optimized performance.<br>

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Load the dataset with engineered features
df = pd.read_csv("engineered_wellbeing_data.csv")

# Select features and target variable
features = ['SOCIAL_ACTIVITY_SCORE', 'WORK_PRODUCTIVITY_SCORE', 'SELF_CARE_SCORE', 'STRESS_IMPACT']
X = df[features]
y = df['WORK_LIFE_BALANCE_ADJUST']  # Target variable (1: Needs Adjustment, 0: Balanced)

# Split the data into training and testing sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Logistic Regression Model (Baseline)
log_model = LogisticRegression()
log_model.fit(X_train, y_train)
log_preds = log_model.predict(X_test)
log_acc = accuracy_score(y_test, log_preds)
print("Logistic Regression Accuracy:", log_acc)

# Train Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_preds = rf_model.predict(X_test)
rf_acc = accuracy_score(y_test, rf_preds)
print("Random Forest Accuracy:", rf_acc)

# Train XGBoost Classifier
xgb_model = XGBClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
xgb_model.fit(X_train, y_train)
xgb_preds = xgb_model.predict(X_test)
xgb_acc = accuracy_score(y_test, xgb_preds)
print("XGBoost Accuracy:", xgb_acc)

# Print Model Performance
print("\nClassification Report (Random Forest):\n", classification_report(y_test, rf_preds))
print("\nClassification Report (XGBoost):\n", classification_report(y_test, xgb_preds))

# Save the trained models
import joblib
joblib.dump(rf_model, "random_forest_work_life_balance.pkl")
joblib.dump(xgb_model, "xgboost_work_life_balance.pkl")

print("Model training complete. Models saved successfully.")


Logistic Regression Accuracy: 0.8624833110814419
Random Forest Accuracy: 0.8648197596795728
XGBoost Accuracy: 0.8621495327102804

Classification Report (Random Forest):
               precision    recall  f1-score   support

           0       0.86      0.87      0.87      1501
           1       0.87      0.86      0.86      1495

    accuracy                           0.86      2996
   macro avg       0.86      0.86      0.86      2996
weighted avg       0.86      0.86      0.86      2996


Classification Report (XGBoost):
               precision    recall  f1-score   support

           0       0.85      0.87      0.86      1501
           1       0.87      0.85      0.86      1495

    accuracy                           0.86      2996
   macro avg       0.86      0.86      0.86      2996
weighted avg       0.86      0.86      0.86      2996

Model training complete. Models saved successfully.


In [ ]:
import joblib
import numpy as np

# Load the trained models
rf_cognitive_model = joblib.load("random_forest_cognitive_overload.pkl")  # Cognitive Overload
rf_social_model = joblib.load("random_forest_social_engagement.pkl")  # Social Engagement Needs
rf_balance_model = joblib.load("random_forest_work_life_balance.pkl")  # Work-Life Balance

# Load the StandardScaler used during training
scaler = joblib.load("scaler.pkl")

# Function to take user input
def get_user_input():
    print("\nEnter your responses to the following questions:")

    # Taking user input for each feature
    social_activity = float(input("Social Activity Score (e.g., 0-10, based on places visited & social engagement): "))
    work_productivity = float(input("Work Productivity Score (e.g., 0-10, based on achievements & task completion): "))
    self_care = float(input("Self-Care Score (e.g., 0-10, based on sleep, meditation, & hobbies): "))
    stress_impact = float(input("Stress Impact Score (e.g., 0-10, considering daily stress & shouting levels): "))

    return np.array([social_activity, work_productivity, self_care, stress_impact]).reshape(1, -1)

# Get user input
user_features = get_user_input()

# Make predictions
cognitive_overload_prediction = rf_cognitive_model.predict(user_features)[0]
social_engagement_prediction = rf_social_model.predict(user_features)[0]
work_life_balance_prediction = rf_balance_model.predict(user_features)[0]

# Output results
print("\n📌 **Predictions Based on Your Input:**")
print("✅ **Cognitive Overload Prediction:**", "YES, you are overloaded 😟" if cognitive_overload_prediction == 1 else "NO, you are not overloaded 😊")
print("✅ **Social Engagement Needs:**", "YES, you need more social engagement 🤝" if social_engagement_prediction == 1 else "NO, you are socially engaged 🎉")
print("✅ **Work-Life Balance Adjustment:**", "YES, you need to adjust your work-life balance ⚖️" if work_life_balance_prediction == 1 else "NO, your work-life balance is good! 💼✨")



Enter your responses to the following questions:
Social Activity Score (e.g., 0-10, based on places visited & social engagement): 5
Work Productivity Score (e.g., 0-10, based on achievements & task completion): 5
Self-Care Score (e.g., 0-10, based on sleep, meditation, & hobbies): 5
Stress Impact Score (e.g., 0-10, considering daily stress & shouting levels): 5

📌 **Predictions Based on Your Input:**
✅ **Cognitive Overload Prediction:** YES, you are overloaded 😟
✅ **Social Engagement Needs:** NO, you are socially engaged 🎉
✅ **Work-Life Balance Adjustment:** NO, your work-life balance is good! 💼✨


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


 Predictive Scheduling with LSTM<br>
Now that we have models to predict cognitive overload, social engagement needs, and work-life balance adjustments, the next step is to build an LSTM (Long Short-Term Memory) model for dynamic scheduling.

This model will:<br> ✅ Analyze user routines over time to suggest the best times for work, socializing, and self-care.<Br>
✅ Predict productivity patterns based on past behavior.<br>
✅ Provide personalized schedule recommendations to optimize daily activities.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

# Load the dataset with engineered features
df = pd.read_csv("engineered_wellbeing_data.csv")

# Define features and target variable for scheduling optimization
features = ['SOCIAL_ACTIVITY_SCORE', 'WORK_PRODUCTIVITY_SCORE', 'SELF_CARE_SCORE', 'STRESS_IMPACT']
target = 'WORK_LIFE_BALANCE_SCORE'  # Predicting the optimal work-life balance score

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df[features])
y = df[target].values.reshape(-1, 1)  # Convert target into a numpy array

# Reshape input for LSTM (samples, time steps, features)
X_lstm = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_lstm, y, test_size=0.2, random_state=42)

# Build LSTM model for scheduling optimization
model = Sequential([
    LSTM(128, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.3),
    LSTM(64),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1)  # Output layer predicting work-life balance score
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

# Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=32, verbose=1)

# Evaluate the model
loss, mae = model.evaluate(X_test, y_test, verbose=1)
print(f"LSTM Model - Mean Absolute Error: {mae}")

# Save the trained model
model.save("lstm_scheduling_model.h5")

print("LSTM scheduling model training complete. Model saved successfully.")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 16s 16ms/step - loss: 0.3837 - mae: 0.4717 - val_loss: 0.1687 - val_mae: 0.3278
Epoch 2/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - loss: 0.1913 - mae: 0.3519 - val_loss: 0.1738 - val_mae: 0.3349
Epoch 3/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 0.1850 - mae: 0.3470 - val_loss: 0.1688 - val_mae: 0.3263
Epoch 4/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.1857 - mae: 0.3462 - val_loss: 0.1733 - val_mae: 0.3340
Epoch 5/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.1840 - mae: 0.3460 - val_loss: 0.1674 - val_mae: 0.3266
Epoch 6/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 0.1857 - mae: 0.3465 - val_loss: 0.1686 - val_mae: 0.3287
Epoch 7/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 0.1871 - mae: 0.3471 - val_loss: 0.1676 - val_mae: 0.3276
Epoch 8/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 0.1837 - mae: 0.3437 - val_loss: 0.1706 - val_mae: 0.3311
Epoch 9/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step 

LSTM Model - Mean Absolute Error: 0.32530203461647034
LSTM scheduling model training complete. Model saved successfully.


Predictive Scheduling with RNN<br>
Now that we've implemented LSTM for scheduling optimization, we will also use RNN (Recurrent Neural Networks) to predict and suggest personalized schedules based on user habits.
<Br>
This RNN model will:<Br> ✅ Analyze time-series patterns in user routines.<br>
✅ Predict productivity levels to optimize task allocation.<br>
✅ Suggest scheduling changes dynamically for improved work-life balance.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

# Load the dataset with engineered features
df = pd.read_csv("engineered_wellbeing_data.csv")

# Define features and target variable for scheduling optimization
features = ['SOCIAL_ACTIVITY_SCORE', 'WORK_PRODUCTIVITY_SCORE', 'SELF_CARE_SCORE', 'STRESS_IMPACT']
target = 'WORK_LIFE_BALANCE_SCORE'  # Predicting the optimal work-life balance score

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df[features])
y = df[target].values.reshape(-1, 1)  # Convert target into a numpy array

# Reshape input for RNN (samples, time steps, features)
X_rnn = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_rnn, y, test_size=0.2, random_state=42)

# Build RNN model for scheduling optimization
model_rnn = Sequential([
    SimpleRNN(128, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.3),
    SimpleRNN(64),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1)  # Output layer predicting work-life balance score
])

# Compile the model
model_rnn.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

# Train the model
model_rnn.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=32, verbose=1)

# Evaluate the model
loss, mae = model_rnn.evaluate(X_test, y_test, verbose=1)
print(f"RNN Model - Mean Absolute Error: {mae}")

# Save the trained model
model_rnn.save("rnn_scheduling_model.h5")

print("RNN scheduling model training complete. Model saved successfully.")


Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


375/375 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - loss: 0.2644 - mae: 0.4055 - val_loss: 0.1766 - val_mae: 0.3378
Epoch 2/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.2014 - mae: 0.3612 - val_loss: 0.1729 - val_mae: 0.3340
Epoch 3/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.1983 - mae: 0.3575 - val_loss: 0.1706 - val_mae: 0.3306
Epoch 4/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.1965 - mae: 0.3543 - val_loss: 0.1710 - val_mae: 0.3284
Epoch 5/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.1934 - mae: 0.3530 - val_loss: 0.1691 - val_mae: 0.3283
Epoch 6/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.1946 - mae: 0.3568 - val_loss: 0.1725 - val_mae: 0.3330
Epoch 7/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.1925 - mae: 0.3550 - val_loss: 0.1689 - val_mae: 0.3273
Epoch 8/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.1916 - mae: 0.3536 - val_loss: 0.1745 - val_mae: 0.3317
Epoch 9/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.1879 

RNN Model - Mean Absolute Error: 0.3268507719039917
RNN scheduling model training complete. Model saved successfully.


In [ ]:
import joblib
import numpy as np
import tensorflow as tf
from tensorflow.keras.losses import MeanSquaredError

# Define custom objects dictionary to include 'mse'
custom_objects = {"mse": MeanSquaredError()}

# Load the trained models with the correct loss function
lstm_model = tf.keras.models.load_model("lstm_scheduling_model.h5", custom_objects=custom_objects)
rnn_model = tf.keras.models.load_model("rnn_scheduling_model.h5", custom_objects=custom_objects)

print("✅ Models loaded successfully!")

# Function to take user input
def get_user_input():
    print("\nEnter your responses to the following questions:")

    social_activity = float(input("Social Activity Score (e.g., 0-10, based on places visited & social engagement): "))
    work_productivity = float(input("Work Productivity Score (e.g., 0-10, based on achievements & task completion): "))
    self_care = float(input("Self-Care Score (e.g., 0-10, based on sleep, meditation, & hobbies): "))
    stress_impact = float(input("Stress Impact Score (e.g., 0-10, considering daily stress & shouting levels): "))

    return np.array([social_activity, work_productivity, self_care, stress_impact]).reshape(1, 1, -1)

# Get user input
user_features = get_user_input()

# Predict with both LSTM and RNN models **without scaling**
lstm_prediction = lstm_model.predict(user_features)[0][0]
rnn_prediction = rnn_model.predict(user_features)[0][0]

# Function to generate schedule recommendations based on raw outputs
def generate_schedule_recommendation(score):
    if score > 8.0:
        return "✅ Your schedule is well-balanced! Maintain current habits. 🏆"
    elif score > 6.0:
        return "🔹 You're productive but need better work-life balance. Try adding more personal time."
    elif score > 4.0:
        return "🛑 Reduce workload in the morning. Schedule rest breaks in the afternoon."
    elif score > 2.0:
        return "⚠️ Your work schedule is too packed! Reduce meetings and take downtime."
    else:
        return "❌ You're overworking! Prioritize self-care and social time."

# Get recommendations
lstm_recommendation = generate_schedule_recommendation(lstm_prediction)
rnn_recommendation = generate_schedule_recommendation(rnn_prediction)

# Output results
print("\n📌 **Predictions Based on Your Input:**")
print(f"🧠 **LSTM Scheduling Suggestion:** {lstm_recommendation}")
print(f"📊 **RNN Scheduling Suggestion:** {rnn_recommendation}")


✅ Models loaded successfully!

Enter your responses to the following questions:
Social Activity Score (e.g., 0-10, based on places visited & social engagement): 8
Work Productivity Score (e.g., 0-10, based on achievements & task completion): 7
Self-Care Score (e.g., 0-10, based on sleep, meditation, & hobbies): 6
Stress Impact Score (e.g., 0-10, considering daily stress & shouting levels): 4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 373ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 309ms/step

📌 **Predictions Based on Your Input:**
🧠 **LSTM Scheduling Suggestion:** 🛑 Reduce workload in the morning. Schedule rest breaks in the afternoon.
📊 **RNN Scheduling Suggestion:** ⚠️ Your work schedule is too packed! Reduce meetings and take downtime.


Reinforcement Learning for Time Optimization<br>
Now that we've implemented LSTM and RNN for predictive scheduling, we will use Reinforcement Learning (RL) to dynamically optimize time management based on real-time user behavior.

This RL model will:<br> ✅ Prioritize tasks dynamically, rewarding timely completion and penalizing missed activities.<br>
✅ Optimize scheduling to improve productivity, social engagement, and self-care balance.<br>
✅ Adapt to user behavior over time, ensuring evolving preferences are considered.

In [ ]:
import numpy as np
import pandas as pd
import random
import joblib

# Load the dataset with engineered features
df = pd.read_csv("engineered_wellbeing_data.csv")

# Define states: User's current work-life balance, social engagement, and stress impact
states = df[['WORK_LIFE_BALANCE_SCORE', 'SOCIAL_ACTIVITY_SCORE', 'STRESS_IMPACT']].values

# Define actions: Task Rescheduling Options (Delay, Prioritize, Maintain)
actions = ['Postpone Task', 'Prioritize Task', 'Maintain Schedule']

# Define the Q-table for Q-learning
q_table = np.zeros((len(states), len(actions)))

# Optimized Hyperparameters
learning_rate = 0.3  # Increase learning speed
discount_factor = 0.95  # Reward future actions more
exploration_rate = 1.0
exploration_decay = 0.99  # Faster decay
min_exploration_rate = 0.05  # Avoid getting stuck in bad policies
episodes = 500  # **Reduced from 5000 to 1000 for faster execution**

# Optimized Q-Learning Algorithm
for episode in range(episodes):
    state_index = random.randint(0, len(states) - 1)  # Start from a random state
    done = False

    while not done:
        # Vectorized Exploration vs Exploitation
        action_index = (
            random.randint(0, len(actions) - 1) if random.uniform(0, 1) < exploration_rate
            else np.argmax(q_table[state_index])
        )

        # Faster Reward Calculation
        reward = (
            -1 if actions[action_index] == 'Postpone Task' and states[state_index][1] > 50 else
            1 if actions[action_index] == 'Prioritize Task' and states[state_index][2] < 30 else
            0.5
        )

        # Move to next state (More random for faster learning)
        next_state_index = random.randint(0, len(states) - 1)

        # Vectorized Q-value Update
        q_table[state_index, action_index] = (
            (1 - learning_rate) * q_table[state_index, action_index]
            + learning_rate * (reward + discount_factor * np.max(q_table[next_state_index]))
        )

        state_index = next_state_index  # Update state

        # Stop early for efficiency
        if episode > episodes - 100:
            done = True

    # Faster exploration decay
    exploration_rate = max(min_exploration_rate, exploration_rate * exploration_decay)

# Save the trained Q-table
joblib.dump(q_table, "q_learning_time_optimization.pkl")

print("🚀 Optimized Q-learning model training complete in ~30 minutes. Model saved successfully.")
